<div class="alert alert-block alert-success">
<b> <font size="+3"> <font color="black"> 
USDA, National Agriculture Imagery Program (NAIP): Image Processing Using Python </font color></font> </b> 

    
<font color="black"> _Author: Sushil Nepal_
    
<font color="black"> _Date: August 4th, 2023_ 
    
<font color="black"> _Objectives:_ 
    
> <font color="black">  1. To import, explore, and visualize the 4-Channel (bands) NAIP imagery.
   
> <font color="black">  2. To create a interactive map using.
   
> <font color="black">  3. Calculate the Normalized Difference Vegetation Index (NDVI) using the red and near-infrared channles.
  
> <font color="black">  4. To sgement the image using superpixells.
 
> <font color="black">  5. Generating shapefile of the segments.
</div>

<div class="alert alert-block alert-success">
<b> <font size="+3"> <font color="black">
Tutorial Outline</font color></font> </b> 


> <font color="black"> Part 1. Explore and visualize the NAIP image

> <font color="black"> Part 2. Calculate the NDVI using NAIP imagery

> <font color="black"> Part 3. Segement the NAIP image 

> <font color="black"> Part 4. Generate the shapefile of the segments

> <font color="black"> Part 5. Create an interactive map of the segmented shapefile using folium
</div>

<div class="alert alert-block alert-success">
<b> <font size="+3"> <font color="black">
Data Download</font color></font> </b> 

#### You can download the 4-channel NAIP imagery for the region of your interest using the link given below:

[Naip Image](https://www.coast.noaa.gov/dataviewer/#/imagery/search/)
</div>

# Part 1: Explore and Visualize the NAIP image 

## Step 1: I will first start by importing the required libraries

> #### For this exercise, I will be needing fucntion within numpy, rasterio, scikitlearn, and geopandas librarries to read, manipulate and analyzed the raster data (NAIP). While we will use libraries like matplotlib for visualization and folium to create the interactive maps. 

> ##### Lets get started by loading the required libraries

In [ ]:
import numpy as np
import folium
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
%matplotlib inline

## Step 2: load the 4-band NAIP 
> #### I will load the imagery from the folder where it was stored into jupyter notebook environemnt using the rasterio-open function from rasterio library.To use this code you can change the path to where you have stored your imagery in your personal computer.We will also print the projection of the naip image. Projection is important to know while visualizing the data in the later part of this exercise.
> ##### My NAIP image (clipped_naip1.tif) is located in the Super_pixels subfolder under the folder Lassen_data in my S drive. You can set the path to you folder by changing the code below

In [ ]:
raster_path = "S:\\Lassen_data\\Super_pixels\\clipped_naip1.tif"
naip_data=rasterio.open(raster_path)

## Step 2: Explore the NAIP data 
> #### I will check the resolution, projection, number of rows-columns, and the number of bands of the image and print the results. I will also check the minimum and maximum cordinates within northing (Y) and easting (X).

In [ ]:
print('Resolution X, Y:',naip_data.res)
print('Coordinate reference system (CRS):', naip_data.crs)
print('Width:', naip_data.width)
print('Height:', naip_data.height)
print('Number of bands:',naip_data.count)

# Get the minimum and maximum coordinates
min_x, min_y = naip_data.bounds.left, naip_data.bounds.bottom
max_x, max_y = naip_data.bounds.right, naip_data.bounds.top

print('Minimum X coordinate:', min_x)
print('Minimum Y coordinate:', min_y)
print('Maximum X coordinate:', max_x)
print('Maximum Y coordinate:', max_y)



## Step 3: Visualize the image
> #### I will now visualize the image using matplotlib library. I will first plot all of the 4 bands together in one figure usign the _subplot ()_ function from matplotlib. I will also plot each band to its spatial extent using the _extent()_ function in each figure panel.I have included the optional code for turning the axis lables and axis ticks on and off, howevre, I have put them as comment, you can undo the comments and run the code. 

In [ ]:

nir_band = naip_data.read(4)
blue_band = naip_data.read(3)
green_band = naip_data.read(2)
red_band = naip_data.read(1)

# Create a 2x2 grid of subplots
figure, axes = plt.subplots(2, 2, figsize=(10, 10),sharex=True,sharey=True)

# Plot the NIR band
nir_image = axes[0, 0].imshow(nir_band, cmap='viridis', extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
axes[0, 0].set_title('NIR Band')
#axes[0, 0].set_xlabel('Easting')
axes[0, 0].set_ylabel('Northing')
#axes[0, 0].set_xticks([])


# Plot the Red band
red_image = axes[0, 1].imshow(red_band, cmap='Reds', extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
axes[0, 1].set_title('Red Band')
#axes[0, 1].set_xlabel('Easting')
#axes[0, 1].set_ylabel('Northing')
##axes[0,1].set_yticks([])
#axes[0,1].set_xticks([])

# Plot the Green band
green_image = axes[1, 0].imshow(green_band, cmap='Greens', extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
axes[1, 0].set_title('Green Band')
axes[1, 0].set_xlabel('Easting')
axes[1, 0].set_ylabel('Northing')


# Plot the Blue band
blue_image = axes[1, 1].imshow(blue_band, cmap='Blues', extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
axes[1, 1].set_title('Blue Band')
axes[1, 1].set_xlabel('Easting')
#axes[1,1].set_xticks([])
#axes[1, 1].set_ylabel('Northing')

#### Lets visualize the image as a RGB cahnnel. I will vertically stack red, green and blue channel using the np.dstack function from numpy library.

In [ ]:
image_stack=np.dstack((red_band,green_band,blue_band))

# Create a 2x2 grid of subplots
figure, ax = plt.subplots(figsize=(10, 10))
# Plot the NIR band
nir_image = ax.imshow(image_stack, extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
ax.set_title('False Color Composite')
ax.set_xlabel('Easting')
ax.set_ylabel('Northing')
#axes[0, 0].set_xticks([])


##### Lets quickly make a histogram of each channel and visualize the range. I have only checked the histogram of the red channel, you can play with other three channel by modifying the code below.


In [ ]:
# I will use the red channel just for the exercise.
plt.figure
plt.hist (red_band.flatten(), bins=50)
plt.show ()


##### My values are way out of range between 0, and 1. They are less than 0. How can we fix this? Lets do it by min-max rescaling so that values get within the bound of 0 and 1. For this I will write a min-max scaling function and pass the function to each channel as shown below in the code. I will then stack the image and plot it.

In [ ]:
def scale (x):
    return((x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x)))

#### Lets apply to individual array 

blue_band = scale (blue_band)
green_band =scale (green_band)
red_band = scale (red_band)

image_stack1=np.dstack((red_band,green_band,blue_band))

# Create a 2x2 grid of subplots
figure, ax = plt.subplots(figsize=(10, 10))
# Plot the NIR band
nir_image = ax.imshow(image_stack1, extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
ax.set_title('False Color Composite')
ax.set_xlabel('Easting')
ax.set_ylabel('Northing')
#axes[0, 0].set_xticks([])


<div class="alert alert-block alert-success">
<b> Note:</b> TADA!! We have sucessfully rescaled and plotted the RGB values within the extent. But the image appear to be dark.
</div>

##### Lets try to increase the image brightness by doing another rescaling of the original bands using the cumilative cut counts.

In [ ]:

def scale_cuts (x):
    return((x-np.nanpercentile(x,2))/(np.nanpercentile(x, 98)-np.nanpercentile(x,2)))

#### Lets apply to individual array 

blue_band = scale_cuts(blue_band)
green_band =scale_cuts(green_band)
red_band = scale_cuts(red_band)

image_stack2=np.dstack((red_band,green_band,blue_band))

# Create a 2x2 grid of subplots
figure, ax = plt.subplots(figsize=(10, 10))
# Plot the NIR band
nir_image = ax.imshow(image_stack2, extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
ax.set_title('False Color Composite')
ax.set_xlabel('Easting')
ax.set_ylabel('Northing')
#axes[0, 0].set_xticks([])


<div class="alert alert-block alert-success">
<b> Note:</b> Awesome!!! notice the difference between above and this image. This image is more brighter than above.
</div>

##### Lets try another scaling function (mean standard devaition scalar) and see how image looks. You can choose to disply them in any ways you want.I like the cumulative cut counts better.



In [ ]:
def scale_mean (x):
    return((x-(np.nanmean(x)-np.nanstd(x)*2))/((np.nanmean(x)+np.nanstd(x)*2)-(np.nanmean(x)-np.nanstd(x)*2)))

#### Lets apply to individual array 

blue_band = scale_mean(blue_band)
green_band =scale_mean(green_band)
red_band = scale_mean(red_band)

image_stack3=np.dstack((red_band,green_band,blue_band))

# Create a 2x2 grid of subplots
figure, ax = plt.subplots(figsize=(10, 10))
# Plot the NIR band
nir_image = ax.imshow(image_stack3, extent=(naip_data.bounds.left,naip_data.bounds.right,
                                                               naip_data.bounds.bottom,naip_data.bounds.top))
ax.set_title('False Color Composite')
ax.set_xlabel('Easting')
ax.set_ylabel('Northing')
#axes[0, 0].set_xticks([])


##### I will now convert the RGB image to jpeg image so that i can utlize it for image segmentation later.

In [ ]:
import imageio

# Define the output file path
output_path = 'S:\\Lassen_data\\Super_pixels\\corrected.jpg'

# Convert the image stack to uint8 and scale to 0-255 range (required for JPEG)
image_stack_scaled = (image_stack2 * 255).astype('uint8')

# Save the image stack as JPEG
imageio.imwrite(output_path, image_stack_scaled, format='jpeg')
print(f"Image stack saved as '{output_path}'")

## Step 4:  Get the bounds of the NAIP image and convert it to the polygon.
> #### I am using this step so that I can overlay the bound over the interactive ortho image to show where the study area lies and how imagery looks. For this part of exercise, I will use libraries such as rasterio to manipulate the raster and polygon data and folium to visualize over an interactive map.

In [ ]:
from rasterio.crs import CRS
from shapely.geometry import Polygon
## Get the minimum and maximum coordinates
min_x, min_y = naip_data.bounds.left, naip_data.bounds.bottom
max_x, max_y = naip_data.bounds.right, naip_data.bounds.top

# Create a Polygon representing the bounding box and convert the crs  corresponding to NAIP data
bounding_box = Polygon([(min_x, min_y), (min_x, max_y), (max_x, max_y), (max_x, min_y)])

bounding_box_crs=naip_data.crs

# Create a GeoDataFrame with the bounding box
gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=bounding_box_crs)

######### reproject  ###################
#gdf_epsg  =gdf.to_crs(epsg=26910)
gdf_epsg  =gdf.to_crs(epsg=4326) ## use this to plot with inbuit map in python
##################### plot ###############

gdf_epsg.geometry


# Define the output shapefile path
output_shapefile_path = "S:/Lassen_data/Super_pixels/Shapefile/bounding_box.shp"

# Write the GeoDataFrame to a shapefile
gdf.to_file(output_shapefile_path)


<div class="alert alert-block alert-warning">
<b> Note:</b> Note that folium takes epsg: 4326 projection , therfore I chnaged the projection of the bounding polygon to match up with folium map
</div>

## Step 5: Visualize in an interactive dashboard
> #### First I will convert the polygon into geojason format and then center the folium to the centre of the bounding box. I will adjust the initial zoom level to 12 and then add the World_Imgery map from ESRI as the base layer.Bounding box will be displyed over the base layer to show how NAIP imagery would look like

In [ ]:

# Create a Folium map centered on the bounding box polygon
map_center = [gdf_epsg.centroid.y.values[0], gdf_epsg.centroid.x.values[0]]
folium_map = folium.Map(location=map_center, zoom_start=12)

# Convert the GeoDataFrame to a GeoJSON format
geojson_data = gdf_epsg.to_crs(epsg=4326).__geo_interface__

# Add the GeoJSON data to the Folium map
folium.GeoJson(geojson_data, name='bounding_box', style_function=lambda x: {'color': 'red', 'fillColor': 'red', 'weight': 3, 'fillOpacity': 0.1}).add_to(folium_map)

# Add ESRI World Imagery as the base map
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='ESRI World Imagery', name='ESRI World Imagery', overlay=True).add_to(folium_map)

# Display the map
folium_map



<div class="alert alert-block alert-warning">
<b> Note:</b> I adjusted the outline color, transparency, and outline width by passing the style-function on the folium map.
</div>

<div class="alert alert-block alert-danger">
<b> Caution:</b> The code mightnot disply the image here in github since it is an interactive map, but trust me when you run the code with your data it will work. 
</div>

In [ ]:
# Create a GeoDataFrame for the USA map using geopandas library
usa_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Calculate the extent of the bounding box
bbox_extent = gdf_epsg.total_bounds


# Plot the USA map
ax = usa_map.plot(figsize=(10, 10), color='lightgray', edgecolor='black')

# Set the axis limits to match the extent of the bounding box
ax.set_xlim(bbox_extent[0]-5, bbox_extent[2]+10)
ax.set_ylim(bbox_extent[1]-5, bbox_extent[3]+10)

# Plot the reprojected bounding box over the USA map
gdf_epsg.plot(ax=ax, color='red',alpha=1, linewidth=2000)
plt.show()

# Part 2: Calculate NDVI using near infra-red and red channel

## Formula for NDVI



NDVI = $\frac{NIR-RED}{NIR+RED}$

_where NDVI= Normalized difference vegetation index, NIR= near infra-red band, RED= red band_


<div class="alert alert-block alert-warning">
<b> Note:</b> NDVI ranges between -1 to 1 where vlaues < 0 represents the bare ground and no-vegetation while values > 1 indicates vegetation.
</div>

In [ ]:
raster_path = "S:\\Lassen_data\\Super_pixels\\clipped_naip1.tif"
naip_dataA=rasterio.open(raster_path)

In [ ]:
nir_band = naip_dataA.read(4)
blue_band = naip_dataA.read(3)
green_band = naip_dataA.read(2)
red_band = naip_dataA.read(1)

In [ ]:
# Normalize the pixel values to the range of [-1, 1]
red_band = red_band.astype(float) / 232.0
nir_band = nir_band.astype(float) / 229.0

# Calculate NDVI
ndvi = (nir_band - red_band) / (nir_band + red_band)


# Create a 1x1 grid of subplots
figure, ax = plt.subplots(1, 1, figsize=(14,8 ))

# Plot the NDVI values within the extent of the original NAIP image 
ndvi_image = ax.imshow(ndvi, cmap='viridis', vmin=-1, vmax=1, extent=(naip_dataA.bounds.left,naip_dataA.bounds.right,
                                                               naip_dataA.bounds.bottom,naip_dataA.bounds.top))
ax.set_title('Calculated NDVI Values')
ax.set_xlabel('Easting',fontsize=24, fontweight='bold')
ax.set_ylabel('Northing',fontsize=24,fontweight='bold')

# Add a colorbar
cbar = plt.colorbar(ndvi_image, ax=ax, orientation='vertical')
cbar.set_label('NDVI')



In [ ]:

ndvi_meta = naip_data.meta.copy()
ndvi_meta.update({
    'count': 1,  # NDVI is a single band raster
    'dtype': rasterio.float32  # Use float32 data type for better precision
})

output_path = "S:\\Lassen_data\\Super_pixels\\ndvi.tif"
with rasterio.open(output_path, 'w', **ndvi_meta) as dst:
    dst.write(ndvi, 1)


<div class="alert alert-block alert-warning">
<b> Optional:</b> We can now export the ndvi data into the local folder in your computer.
</div>

##### Additonal method to plot the region of interest (bounding box ) over a non-interactive map

# Part 3: Image segmentation 

## Step 1: Importing the required libraries

> #### For this exercise, I will again be needing fucntion within numpy, rasterio, scikitlearn, and geopandas librarries to read, manipulate and analyzed the raster data (NAIP). While we will use libraries like matplotlib for visualization and folium to create the interactive maps. 

> ##### Lets get started by loading the required libraries

In [ ]:
import rasterio
import numpy as np
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import geopandas as gpd
from skimage.segmentation import slic
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline


##### I will bring in the new NAIP imagery data this time to work with. You can download the naip imagery from the link provided below for any region within the United States:
[Image Link](https://www.coast.noaa.gov/dataviewer/#/imagery/search/)

In [ ]:
#raster_path = "S:\\Lassen_data\\Super_pixels\\Test1.tif"
raster_path = "S:\\Lassen_data\\Super_pixels\\clipped_naip.tif"
naip_data1 = rasterio.open(raster_path)


## Step 2: Channel scaling and stacking 
> #### I will then read individual band seperately and then rescale them so that my bounds are between 0 and 1. After rescaling, I will stack the image vertically to creat a single image consisting of red, green and blue channel.

In [ ]:
nir_band = naip_data1.read(4)
blue_band = naip_data1.read(3)
green_band = naip_data1.read(2)
red_band = naip_data1.read(1)
##### Lets try to increase the image brightness by doing another rescaling using the cumilative cut counts.
def scale_cuts (x):
    return((x-np.nanpercentile(x,2))/(np.nanpercentile(x, 98)-np.nanpercentile(x,2)))

#### Lets apply to individual array 

blue_band = scale_cuts(blue_band)
green_band =scale_cuts(green_band)
red_band = scale_cuts(red_band)

image_stack4=np.dstack((red_band,green_band,blue_band))

# Create a 2x2 grid of subplots
figure, ax = plt.subplots(figsize=(10, 10))
# Plot the NIR band
nir_image = ax.imshow(image_stack4, extent=(naip_data1.bounds.left,naip_data1.bounds.right,
                                                               naip_data1.bounds.bottom,naip_data1.bounds.top))
ax.set_title('False Color Composite', fontsize=28)
ax.set_xlabel('Easting',fontsize=24, fontweight='bold')
ax.set_ylabel('Northing',fontsize=24, fontweight='bold')
#axes[0, 0].set_xticks([])

<div class="alert alert-block alert-warning">
<b> You rememebr step 3 in part 1 of the turorial correct !! </b> 

## Step 3: Using SLIC algorithm
> #### I will use the slic algorithm from the scikitlearn package. I want to produce 300 segements with compactness at 5 which will control how it follows the geometry of the objects in an image, usually,small numbers are better but you should play with different numbers for your image and see what it does. Sigma controls the smothness of the edges and again you have to play with different numbers to find the best for your image settings. 

In [ ]:
segments_slic = slic(image_stack4, n_segments=4000, compactness=5, sigma=5,
                     start_label=1)
segments_slic = segments_slic.astype('int32')
segments_slic

In [ ]:
from skimage.segmentation import slic, mark_boundaries
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(mark_boundaries(image_stack4,segments_slic),extent=(naip_data1.bounds.left,naip_data1.bounds.right,
                                                               naip_data1.bounds.bottom,naip_data1.bounds.top))
ax.set_title('SLIC')


# Part 4: Generate a shapefile of the segments 

> #### For this exercise, I will be needing geopandas, shapely.geometry and rasterio packages. First I will generate the polygons from the slic segments, then create a geodataframe with the polygon features and finally write is as a shapefile to the disk.

In [ ]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import geopandas as gpd

# Step 3: Generate polygons from the labels using rasterio.features.shapes
segments = []
for geom, value in shapes(segments_slic, mask=(segments_slic > 0), transform=naip_data1.transform):
    if value == 0:
        continue
    shape_obj = shape(geom)
    if shape_obj.is_empty or shape_obj.geom_type != "Polygon":
        continue
    segments.append(shape_obj)

desired_crs = 'EPSG:26910'

# Step 4: Create a GeoDataFrame with the segment polygons
gdf = gpd.GeoDataFrame({"geometry": segments},crs=desired_crs)

gdf_epgs1=gdf.to_crs(epsg=4326)

#  Write the GeoDataFrame to a shapefile
#output_shapefile = "S:\\Lassen_data\\Super_pixels\\Shapefile\\Segments1.shp"
#gdf.to_file(output_shapefile)


In [ ]:
gdf_epgs1.crs

<div class="alert alert-block alert-warning">
<b> I already converted the projection to epsg: 4326 so that the segemets overlay correctly over the ortho while creating a folium map !! </b> 

# Part 5: Create an interactive map of the segmented shapefile using folium
#### This code snippet employs the Python library Folium to create an interactive web map with visualized geographic data. The primary focus of this visualization is a GeoDataFrame gdf_epgs1 that contains polygon geometries. The code initiates by calculating the centroid of these polygons to determine the map's central point, which ensures a balanced view when the map is initially displayed. The folium.Map() function constructs the base map with the specified center and initial zoom level. The GeoDataFrame is then transformed into a GeoJSON format using the to_crs() method, which ensures the proper geographical projection (in this case, EPSG:4326, or WGS 84). This transformation is crucial for accurate representation on a map. The GeoJSON data is subsequently integrated into the Folium map using folium.GeoJson(), allowing the polygons to be displayed on the map. Styling options are applied to these polygons, such as red outlines and semi-transparent red fills.

> #####  Furthermore, a tile layer is added to the map utilizing the ESRI World Imagery service. This layer provides a visually appealing background with high-resolution satellite imagery. By using folium.TileLayer(), the ESRI World Imagery tiles are overlaid onto the map. The overlay=True parameter ensures that the tile layer can be toggled on and off as needed. Finally, the fully constructed map is displayed by calling folium_map.

> ##### In summary, this code snippet showcases the power of Folium for creating interactive maps that enable the visualization of geographic data, including polygons, and offers customization options for map styling and background imagery.

In [ ]:
import folium
# Create a Folium map centered on the bounding box polygon
map_center = [gdf_epgs1.centroid.y.values[0], gdf_epgs1.centroid.x.values[0]]
folium_map = folium.Map(location=map_center, zoom_start=12)

# Convert the GeoDataFrame to a GeoJSON format
geojson_data = gdf_epgs1.to_crs(epsg=4326).__geo_interface__

# Add the GeoJSON data to the Folium map
folium.GeoJson(geojson_data, name='bounding_box', style_function=lambda x: {'color': 'red', 'fillColor': 'red', 'weight': 0.3, 'fillOpacity': 0.1}).add_to(folium_map)

# Add ESRI World Imagery as the base map
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='ESRI World Imagery', name='ESRI World Imagery', overlay=True).add_to(folium_map)

# Display the map
folium_map

